### Stage 1 : Defining IfcPropertySetTemplate & IfcPropertyTemplate

In [ ]:
import ifcopenshell
import ifcopenshell.util.element
import ifcopenshell.guid as create_guid
from ifcopenshell import api

# Open the IFC file
ifc_file = ifcopenshell.open("../Models/Duplex_House.ifc")

# Function to create a property set template (Pset)
def create_property_set_template(ifc_file, name, description):
    return ifc_file.createIfcPropertySetTemplate(
        GlobalId=create_guid.new(),
        OwnerHistory=None,
        Name=name,
        Description=description,
        TemplateType="PSET_TYPEDRIVENONLY",
        ApplicableEntity="IfcObject"
    )

# Function to create a property template
def create_property_template(ifc_file, name, description, primary_measure_type):
    return ifc_file.createIfcSimplePropertyTemplate(
        GlobalId=create_guid.new(),
        Name=name,
        Description=description,
        PrimaryMeasureType=primary_measure_type,
        TemplateType="P_SINGLEVALUE"
    )

# Create property set templates
taskcommon = create_property_set_template(ifc_file, "Pset_TaskCommon", "Task Related Properties")
tasktimecommon = create_property_set_template(ifc_file, "Pset_TaskTimeCommon", "Task Related Properties")
resourcecommon = create_property_set_template(ifc_file, "Pset_ResourceCommon", "Resource Related Properties")
costcommon = create_property_set_template(ifc_file, "Pset_CostCommon", "Cost Related Properties")
planned_resource = create_property_set_template(ifc_file, "Pset_PlannedResourceQuantity", "Planned resource values")
actual_resource = create_property_set_template(ifc_file, "Pset_ActualResourceQuantity", "Actual resource values")

# Define property lists
task_properties_list = [
    ("TaskId", "Unique identifier for the task.", "IfcLabel"),
    ("TaskName", "Name of the task.", "IfcLabel"),
    ("TaskDescription", "Detailed description of the task.", "IfcText"),
    ("IsCritical", "Indicates whether the task is critical to the project.", "IfcBoolean"),
    ("Status", "Current status of the task.", "IfcLabel"),
    ("DelayReason", "Reason for any delay in task completion.", "IfcLabel"),
]

task_time_properties_list = [
    ("PlannedStartDate", "Scheduled start date of the task.", "IfcDate"),
    ("PlannedEndDate", "Scheduled end date of the task.", "IfcDate"),
    ("PlannedDuration", "Estimated duration of the task in days.", "IfcReal"),
    ("ActualStartDate", "Actual start date of the task.", "IfcDate"),
    ("ActualEndDate", "Actual end date of the task.", "IfcDate"),
    ("ActualDuration", "Actual duration of the task in days.", "IfcReal"),
]

cost_properties_list = [
    ("PlannedLaborCost", "Estimated labor cost for the task.", "IfcReal"),
    ("PlannedEquipmentCost", "Estimated equipment cost for the task.", "IfcReal"),
    ("PlannedMaterialCost", "Estimated material cost for the task.", "IfcReal"),
    ("PlannedExpense", "Estimated miscellaneous expenses for the task.", "IfcReal"),
    ("PlannedTotalCost", "Total estimated cost for the task.", "IfcReal"),
    ("ActualLaborCost", "Actual labor cost incurred for the task.", "IfcReal"),
    ("ActualEquipmentCost", "Actual equipment cost incurred for the task.", "IfcReal"),
    ("ActualMaterialCost", "Actual material cost incurred for the task.", "IfcReal"),
    ("ActualExpense", "Actual miscellaneous expenses for the task.", "IfcReal"),
    ("ActualTotalCost", "Total actual cost incurred for the task.", "IfcReal"),
]

planned_resource_properties_list = [
    ("Concrete", "Resource Description", "IfcReal"),
    ("Steel", "Resource Description", "IfcReal"),
    ("Brick", "Resource Description", "IfcReal"),
    ("ConcreteVibrator", "Resource Description", "IfcReal"),
    ("ConcretePump", "Resource Description", "IfcReal"),
    ("Helper", "Resource Description", "IfcReal"),
    ("Plumber", "Resource Description", "IfcReal"),
    ("Carpenter", "Resource Description", "IfcReal"),
    ("Ironworker", "Resource Description", "IfcReal"),
]

actual_resource_properties_list = planned_resource_properties_list.copy()

# Function to create and assign property templates
def assign_properties_to_pset(ifc_file, pset, properties_list):
    properties = [create_property_template(ifc_file, name, description, measure) for name, description, measure in properties_list]
    pset.HasPropertyTemplates = properties

# Assign properties to property sets
assign_properties_to_pset(ifc_file, taskcommon, task_properties_list)
assign_properties_to_pset(ifc_file, tasktimecommon, task_time_properties_list)
assign_properties_to_pset(ifc_file, costcommon, cost_properties_list)
assign_properties_to_pset(ifc_file, planned_resource, planned_resource_properties_list)
assign_properties_to_pset(ifc_file, actual_resource, actual_resource_properties_list)


In [ ]:
# Save the modified IFC file
ifc_file.write("../Models/Duplex_House_Modified.ifc")

In [ ]:
import ifcopenshell
from tabulate import tabulate

# Open the IFC file
ifc_file = ifcopenshell.open("../Models/Duplex_House_Modified.ifc")

# Function to access and return properties in a Pset template
def get_pset_properties(pset_template):
    properties_table = []
    
    # Check if HasPropertyTemplates is not None
    if pset_template.HasPropertyTemplates:
        for prop in pset_template.HasPropertyTemplates:
            properties_table.append([prop.Name, prop.Description, prop.PrimaryMeasureType])
    else:
        properties_table.append(["No properties assigned", "-", "-"])
    
    return properties_table

# Access specific PsetTemplates by name
pset_templates = [pset for pset in ifc_file.by_type("IfcPropertySetTemplate")]

# Print properties for each PsetTemplate in tabular form
for pset_template in pset_templates:
    print(f"Property Set Template: {pset_template.Name}")
    properties_table = get_pset_properties(pset_template)
    print(tabulate(properties_table, headers=["Property Name", "Description", "Measure Type"], tablefmt="grid"))
    print()

### Stage 2: Adding Inforamtion to Model

In [ ]:
building_element = ifc_file.by_guid("2FUQJjpVj9wBkoUxY59XRk")
building_element_info = ifcopenshell.util.element.get_psets(building_element)
building_element_info

In [ ]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.error import URLError

# Use the actual SPARQL endpoint URL
# This should point to your running Fuseki server
sparql = SPARQLWrapper("http://localhost:3030/bimontoDB/sparql")

# SPARQL query to retrieve building element and associated task details.
sparql.setQuery("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX bimonto: <http://w3id.org/IproK/00/BIMOnto#>
PREFIX iprok: <http://w3id.org/IproK#>

SELECT ?Element_Name ?Task_Name ?Schedule_Start ?Actual_Start ?Resource ?Budgeted_Units
WHERE {
  ?building_element bimonto:GlobalId "2FUQJjpVj9wBkoUxY59XRk" ;
                    bimonto:Name ?Element_Name ;
                    bimonto:AssignToProcess ?assignedTask .

  ?assignedTask     iprok:hasTaskTime ?task_time_info ;
                    iprok:requiresResource ?resource_info ;
                    iprok:Name ?Task_Name .

  ?task_time_info   iprok:ScheduleStart ?Schedule_Start ;
                    iprok:ActualStart ?Actual_Start .

  ?resource_info    rdf:type ?Resource ;
                    iprok:BudgetedUnits ?Budgeted_Units .

  FILTER (?Resource != owl:NamedIndividual)
}
""")

# Set the expected return format to JSON
sparql.setReturnFormat(JSON)

try:
    # Execute the query and convert the results
    results = sparql.query().convert()

    # Convert the results dictionary to a clean, indented JSON string and print it
    print(json.dumps(results, indent=4))

except URLError as e:
    # Create a JSON error object and print it
    error_output = {
        "error": "ConnectionError",
        "message": f"Error connecting to SPARQL endpoint: {e}"
    }
    print(json.dumps(error_output, indent=4))
except Exception as e:
    # Create a JSON error object and print it
    error_output = {
        "error": "ExecutionError",
        "message": f"An error occurred: {e}"
    }
    print(json.dumps(error_output, indent=4))

In [ ]:
import ifcopenshell
import ifcopenshell.api
import ifcopenshell.guid
import json
import os

# --- 1. JSON Data from your SPARQL Query ---
json_data_string = """
{
    "head": {
        "vars": [
            "Element_Name",
            "Task_Name",
            "Schedule_Start",
            "Actual_Start",
            "Resource",
            "Budgeted_Units"
        ]
    },
    "results": {
        "bindings": [
            {"Element_Name": {"type": "literal", "value": "Concrete-Rectangular Beam:PBEAM_230X300:367477"}, "Task_Name": {"type": "literal", "value": "Slab and Beam Concrete Casting"}, "Schedule_Start": {"type": "literal", "value": "04-09-2023"}, "Actual_Start": {"type": "literal", "value": "08-09-2023"}, "Resource": {"type": "uri", "value": "http://w3id.org/IproK#ConcretePump"}, "Budgeted_Units": {"type": "literal", "datatype": "http://www.w3.org/2001/XMLSchema#decimal", "value": "1.0"}},
            {"Element_Name": {"type": "literal", "value": "Concrete-Rectangular Beam:PBEAM_230X300:367477"}, "Task_Name": {"type": "literal", "value": "Slab and Beam Concrete Casting"}, "Schedule_Start": {"type": "literal", "value": "04-09-2023"}, "Actual_Start": {"type": "literal", "value": "08-09-2023"}, "Resource": {"type": "uri", "value": "http://w3id.org/IproK#Concrete"}, "Budgeted_Units": {"type": "literal", "datatype": "http://www.w3.org/2001/XMLSchema#decimal", "value": "30.0"}},
            {"Element_Name": {"type": "literal", "value": "Concrete-Rectangular Beam:PBEAM_230X300:367477"}, "Task_Name": {"type": "literal", "value": "Slab and Beam Concrete Casting"}, "Schedule_Start": {"type": "literal", "value": "04-09-2023"}, "Actual_Start": {"type": "literal", "value": "08-09-2023"}, "Resource": {"type": "uri", "value": "http://w3id.org/IproK#Helper"}, "Budgeted_Units": {"type": "literal", "datatype": "http://www.w3.org/2001/XMLSchema#decimal", "value": "2.0"}},
            {"Element_Name": {"type": "literal", "value": "Concrete-Rectangular Beam:PBEAM_230X300:367477"}, "Task_Name": {"type": "literal", "value": "Slab and Beam Concrete Casting"}, "Schedule_Start": {"type": "literal", "value": "04-09-2023"}, "Actual_Start": {"type": "literal", "value": "08-09-2023"}, "Resource": {"type": "uri", "value": "http://w3id.org/IproK#ConcreteVibrator"}, "Budgeted_Units": {"type": "literal", "datatype": "http://www.w3.org/2001/XMLSchema#decimal", "value": "2.0"}},
            {"Element_Name": {"type": "literal", "value": "Concrete-Rectangular Beam:PBEAM_230X300:367477"}, "Task_Name": {"type": "literal", "value": "Slab and Beam Concrete Casting"}, "Schedule_Start": {"type": "literal", "value": "04-09-2023"}, "Actual_Start": {"type": "literal", "value": "08-09-2023"}, "Resource": {"type": "uri", "value": "http://w3id.org/IproK#ConcreteMixers"}, "Budgeted_Units": {"type": "literal", "datatype": "http://www.w3.org/2001/XMLSchema#decimal", "value": "1.0"}},
            {"Element_Name": {"type": "literal", "value": "Concrete-Rectangular Beam:PBEAM_230X300:367477"}, "Task_Name": {"type": "literal", "value": "Slab and Beam Formwork"}, "Schedule_Start": {"type": "literal", "value": "27-08-2023"}, "Actual_Start": {"type": "literal", "value": "29-08-2023"}, "Resource": {"type": "uri", "value": "http://w3id.org/IproK#FormWork"}, "Budgeted_Units": {"type": "literal", "datatype": "http://www.w3.org/2001/XMLSchema#decimal", "value": "50.0"}},
            {"Element_Name": {"type": "literal", "value": "Concrete-Rectangular Beam:PBEAM_230X300:367477"}, "Task_Name": {"type": "literal", "value": "Slab and Beam Formwork"}, "Schedule_Start": {"type": "literal", "value": "27-08-2023"}, "Actual_Start": {"type": "literal", "value": "29-08-2023"}, "Resource": {"type": "uri", "value": "http://w3id.org/IproK#Helper"}, "Budgeted_Units": {"type": "literal", "datatype": "http://www.w3.org/2001/XMLSchema#decimal", "value": "3.0"}},
            {"Element_Name": {"type": "literal", "value": "Concrete-Rectangular Beam:PBEAM_230X300:367477"}, "Task_Name": {"type": "literal", "value": "Slab and Beam Formwork"}, "Schedule_Start": {"type": "literal", "value": "27-08-2023"}, "Actual_Start": {"type": "literal", "value": "29-08-2023"}, "Resource": {"type": "uri", "value": "http://w3id.org/IproK#Carpenter"}, "Budgeted_Units": {"type": "literal", "datatype": "http://www.w3.org/2001/XMLSchema#decimal", "value": "2.0"}},
            {"Element_Name": {"type": "literal", "value": "Concrete-Rectangular Beam:PBEAM_230X300:367477"}, "Task_Name": {"type": "literal", "value": "Slab and Beam Steel Reinforcement"}, "Schedule_Start": {"type": "literal", "value": "29-08-2023"}, "Actual_Start": {"type": "literal", "value": "03-09-2023"}, "Resource": {"type": "uri", "value": "http://w3id.org/IproK#BindingWire"}, "Budgeted_Units": {"type": "literal", "datatype": "http://www.w3.org/2001/XMLSchema#decimal", "value": "10.0"}},
            {"Element_Name": {"type": "literal", "value": "Concrete-Rectangular Beam:PBEAM_230X300:367477"}, "Task_Name": {"type": "literal", "value": "Slab and Beam Steel Reinforcement"}, "Schedule_Start": {"type": "literal", "value": "29-08-2023"}, "Actual_Start": {"type": "literal", "value": "03-09-2023"}, "Resource": {"type": "uri", "value": "http://w3id.org/IproK#Ironworker"}, "Budgeted_Units": {"type": "literal", "datatype": "http://www.w3.org/2001/XMLSchema#decimal", "value": "2.0"}},
            {"Element_Name": {"type": "literal", "value": "Concrete-Rectangular Beam:PBEAM_230X300:367477"}, "Task_Name": {"type": "literal", "value": "Slab and Beam Steel Reinforcement"}, "Schedule_Start": {"type": "literal", "value": "29-08-2023"}, "Actual_Start": {"type": "literal", "value": "03-09-2023"}, "Resource": {"type": "uri", "value": "http://w3id.org/IproK#Helper"}, "Budgeted_Units": {"type": "literal", "datatype": "http://www.w3.org/2001/XMLSchema#decimal", "value": "4.0"}},
            {"Element_Name": {"type": "literal", "value": "Concrete-Rectangular Beam:PBEAM_230X300:367477"}, "Task_Name": {"type": "literal", "value": "Slab and Beam Curring"}, "Schedule_Start": {"type": "literal", "value": "05-09-2023"}, "Actual_Start": {"type": "literal", "value": "10-09-2023"}, "Resource": {"type": "uri", "value": "http://w3id.org/IproK#Helper"}, "Budgeted_Units": {"type": "literal", "datatype": "http://www.w3.org/2001/XMLSchema#decimal", "value": "2.0"}}
        ]
    }
}
"""

def process_sparql_results(data):
    bindings = data["results"]["bindings"]
    tasks = {}
    for item in bindings:
        task_name = item["Task_Name"]["value"]
        if task_name not in tasks:
            tasks[task_name] = {
                "PlannedStartDate": item["Schedule_Start"]["value"],
                "ActualStartDate": item["Actual_Start"]["value"],
                "Resources": {}
            }
        resource_uri = item["Resource"]["value"]
        resource_name = resource_uri.split('#')[-1]
        budgeted_units = float(item["Budgeted_Units"]["value"])
        tasks[task_name]["Resources"][resource_name] = budgeted_units
    return tasks

def main():
    # --- Configuration ---
    input_ifc_path = "../Models/Duplex_House_Modified.ifc"
    output_ifc_path = "../Models/Duplex_House_With_Tasks.ifc"
    building_element_guid = "2FUQJjpVj9wBkoUxY59XRk"

    if not os.path.exists(input_ifc_path):
        print(f"Error: Input IFC file not found at '{input_ifc_path}'")
        return

    # --- Process the JSON data ---
    sparql_results = json.loads(json_data_string)
    grouped_tasks = process_sparql_results(sparql_results)
    
    # --- Load IFC File and Element ---
    ifc_file = ifcopenshell.open(input_ifc_path)
    building_element = ifc_file.by_guid(building_element_guid)

    if not building_element:
        print(f"Error: Element with GUID {building_element_guid} not found.")
        return
        
    # --- Get Pset Templates from the IFC file ---
    pset_template_task = next((t for t in ifc_file.by_type("IfcPropertySetTemplate") if t.Name == "Pset_TaskCommon"), None)
    pset_template_time = next((t for t in ifc_file.by_type("IfcPropertySetTemplate") if t.Name == "Pset_TaskTimeCommon"), None)
    pset_template_resource = next((t for t in ifc_file.by_type("IfcPropertySetTemplate") if t.Name == "Pset_PlannedResourceQuantity"), None)
    
    if not all([pset_template_task, pset_template_time, pset_template_resource]):
        print("Error: One or more required IfcPropertySetTemplates are missing from the IFC file.")
        return

    # --- Create IfcTask and associated Psets for each task ---
    print(f"Found element: {building_element.Name}. Adding tasks...")
    for task_name, task_data in grouped_tasks.items():
        print(f"\nProcessing Task: {task_name}")
        
        task = ifc_file.create_entity("IfcTask", GlobalId=ifcopenshell.guid.new(), Name=task_name)
        print(f"  - Created IfcTask with ID: {task.id()}")

        # ifcopenshell.api.run("project.assign_product", ifc_file, product=task, relating_object=building_element)

        pset_task = ifcopenshell.api.run("pset.add_pset", ifc_file, product=task, name="Pset_TaskCommon")
        ifcopenshell.api.run("pset.edit_pset", ifc_file, pset=pset_task, properties={"TaskName": task_name}, pset_template=pset_template_task)
        print("  - Added and populated Pset_TaskCommon")

        pset_time = ifcopenshell.api.run("pset.add_pset", ifc_file, product=task, name="Pset_TaskTimeCommon")
        ifcopenshell.api.run("pset.edit_pset", ifc_file, pset=pset_time, properties={
            "PlannedStartDate": task_data["PlannedStartDate"],
            "ActualStartDate": task_data["ActualStartDate"]
        }, pset_template=pset_template_time)
        print("  - Added and populated Pset_TaskTimeCommon")
        
        # --- ROBUSTNESS IMPROVEMENT ---
        # 1. Get the list of valid property names from the resource template
        valid_resource_names = {p.Name for p in pset_template_resource.HasPropertyTemplates}

        # 2. Filter the incoming resources to only include those that exist in the template
        resource_properties_to_add = {
            name: value
            for name, value in task_data["Resources"].items()
            if name in valid_resource_names
        }

        # 3. Warn the user about any resources that were skipped
        skipped_resources = set(task_data["Resources"].keys()) - set(resource_properties_to_add.keys())
        if skipped_resources:
            print(f"  - WARNING: Skipped resources not in template: {', '.join(skipped_resources)}")
        
        # 4. Add the pset and populate it with the filtered, valid properties
        pset_resource = ifcopenshell.api.run("pset.add_pset", ifc_file, product=task, name="Pset_PlannedResourceQuantity")
        ifcopenshell.api.run(
            "pset.edit_pset",
            ifc_file,
            pset=pset_resource,
            properties=resource_properties_to_add, # Use the filtered dictionary
            pset_template=pset_template_resource
        )
        print("  - Added and populated Pset_PlannedResourceQuantity")

    # --- Save the modified IFC file ---
    ifc_file.write(output_ifc_path)
    print(f"\nSuccessfully wrote updated IFC file to: {output_ifc_path}")

if __name__ == "__main__":
    main()

In [ ]:
# Save the modified IFC file
ifc_file.write("../Models/Duplex_House_Modified.ifc")

In [ ]:
# Open the IFC file
ifc_file = ifcopenshell.open("../Models/Duplex_House_Modified.ifc")

In [ ]:
building_element = ifc_file.by_guid("2FUQJjpVj9wBkoUxY59XRk")
building_element_info = ifcopenshell.util.element.get_psets(building_element, psets_only=True)
building_element_info

In [ ]:
import ifcopenshell
import ifcopenshell.util.element
import json
import os

# --- Verification Script ---
# This script opens the output file and shows the psets on the tasks that
# have been related to the building element.

output_ifc_path = "../Models/Duplex_House_With_Tasks.ifc"
building_element_guid = "2FUQJjpVj9wBkoUxY59XRk"

if not os.path.exists(output_ifc_path):
    print(f"Error: Output IFC file not found at '{output_ifc_path}'")
else:
    ifc_file = ifcopenshell.open(output_ifc_path)
    building_element = ifc_file.by_guid(building_element_guid)

    print(f"--- Verifying data for element: {building_element.Name} ({building_element.GlobalId}) ---\n")

    # The get_psets function on the element itself will only show its direct psets.
    # The task psets will NOT be listed here.
    print("--- 1. Property Sets directly on the Beam (Original Psets) ---")
    building_element_info = ifcopenshell.util.element.get_psets(building_element, psets_only=True)
    print(json.dumps(building_element_info, indent=2))
    print("\n" + "="*70 + "\n")


    # To find the new data, we must look at the tasks assigned to the beam.
    print("--- 2. Traversing relationship to find Property Sets on related IfcTasks ---")

    # 'HasAssignments' is an inverse attribute that finds all assignment relationships
    # pointing to this building element.
    if not hasattr(building_element, "HasAssignments") or not building_element.HasAssignments:
        print("No tasks assigned to this element.")
    else:
        # Iterate through all assignment relationships for the beam
        for rel_assigns in building_element.HasAssignments:
            # We are interested in the relationship that links to a Process (like IfcTask)
            if rel_assigns.is_a("IfcRelAssignsToProduct"):
                # The task is the 'RelatingProcess' in this relationship
                task = rel_assigns.RelatingProcess
                if task.is_a("IfcTask"):
                    print(f"\n>>> Found related task: '{task.Name}' (ID: {task.id()})")
                    # Now, get the Psets for this specific task
                    task_psets = ifcopenshell.util.element.get_psets(task, psets_only=True)
                    print(json.dumps(task_psets, indent=2, sort_keys=True))

In [23]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.error import URLError

def run_sparql_query(building_element_guid):
    """
    Queries the ontology for all data related to a specific building element GUID.
    """
    sparql = SPARQLWrapper("http://localhost:3030/bimontoDB/sparql")
    
    # This comprehensive query fetches all related task, time, resource, and cost data.
    # OPTIONAL blocks are used because a task may not have all types of data.
    query = f"""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX bimonto: <http://w3id.org/IproK/00/BIMOnto#>
        PREFIX iprok: <http://w3id.org/IproK#>

        SELECT *
        WHERE {{
          ?building_element bimonto:GlobalId "{building_element_guid}" ;
                            bimonto:AssignToProcess ?task .
          
          ?task iprok:Name ?TaskName .
          
          OPTIONAL {{
            ?task iprok:hasTaskTime ?task_time .
            ?task_time iprok:ScheduleStart ?ScheduleStart ;
                       iprok:ScheduleFinish ?ScheduleFinish ;
                       iprok:ScheduleDuration ?ScheduleDuration ;
                       iprok:ActualStart ?ActualStart ;
                       iprok:ActualFinish ?ActualFinish ;
                       iprok:ActualDuration ?ActualDuration .
          }}
          
          OPTIONAL {{
            ?task iprok:requiresResource ?resource .
            ?resource rdf:type ?ResourceType ;
                      iprok:BudgetedUnits ?BudgetedUnits ;
                      iprok:ActualUnits ?ActualUnits .
          }}
          
          OPTIONAL {{
            ?task iprok:hasCostItem ?cost_item .
            ?cost_item iprok:hasCostType ?CostType ;;
                       iprok:ActualCost ?ActualCost ;
                       iprok:BudgetedCost ?BudgetedCost .
                   
          }}
        }}
    """
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        print("Querying SPARQL endpoint...")
        results = sparql.query().convert()
        print("Query successful.")
        return results["results"]["bindings"]
    except URLError as e:
        print(f"Error connecting to SPARQL endpoint: {e}")
    except Exception as e:
        print(f"An error occurred during SPARQL query: {e}")
    return None

In [24]:
run_sparql_query("2FUQJjpVj9wBkoUxY59XRk")

Querying SPARQL endpoint...
Query successful.


[{'building_element': {'type': 'uri',
   'value': 'http://w3id.org/IproK/00/BIMOnto#2FUQJjpVj9wBkoUxY59XRk'},
  'task': {'type': 'uri', 'value': 'http://w3id.org/IproK#TA-ST-GF-0030'},
  'TaskName': {'type': 'literal', 'value': 'Slab and Beam Concrete Casting'},
  'task_time': {'type': 'uri', 'value': 'http://w3id.org/IproK#TT-ST-GF-0030'},
  'ScheduleStart': {'type': 'literal', 'value': '04-09-2023'},
  'ScheduleFinish': {'type': 'literal', 'value': '04-09-2023'},
  'ScheduleDuration': {'type': 'literal',
   'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
   'value': '1.0'},
  'ActualStart': {'type': 'literal', 'value': '08-09-2023'},
  'ActualFinish': {'type': 'literal', 'value': '13-09-2023'},
  'ActualDuration': {'type': 'literal',
   'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
   'value': '6.0'},
  'resource': {'type': 'uri', 'value': 'http://w3id.org/IproK#RS-CC_4f63b'},
  'ResourceType': {'type': 'uri', 'value': 'http://w3id.org/IproK#Concrete'},
  'BudgetedU